<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192Win_context_Weighted_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

fatal: destination path 'idl-spring-22-project-deepar' already exists and is not an empty directory.


In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
# !python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

In [4]:
!mkdir data

In [5]:
!unzip /content/drive/MyDrive/DeepAR_Data/varun_data/company_data.zip -d /content/data

Archive:  /content/drive/MyDrive/DeepAR_Data/varun_data/company_data.zip
   creating: /content/data/content/idl-spring-22-project-deepar/data/
  inflating: /content/data/content/idl-spring-22-project-deepar/data/stock_inputs.npy  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/volume.png  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/stock_test_labels.npy  
   creating: /content/data/content/idl-spring-22-project-deepar/data/company_data/
  inflating: /content/data/content/idl-spring-22-project-deepar/data/company_data/ENS.csv  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/company_data/PSTH.csv  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/company_data/QS.csv  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/company_data/CLPT.csv  
  inflating: /content/data/content/idl-spring-22-project-deepar/data/company_data/JUNO.csv  
  inflating: /content/data/content/idl-spring-22-

In [6]:
import pickle
with open('/content/data/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/data/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [7]:
print(index_to_company)
print(company_to_index)

{0: 'FCX', 1: 'FTCH', 2: 'TGT', 3: 'CRWD', 4: 'CRON', 5: 'MGM', 6: 'CRSM', 7: 'PRU', 8: 'CLOV', 9: 'BTCUSD', 10: 'NEM', 11: 'FXE', 12: 'VRM', 13: 'TER', 14: 'WCLD', 15: 'AVCT', 16: 'GOLD', 17: 'NICH', 18: 'MMM', 19: 'VUZI', 20: 'LMND', 21: 'AMGN', 22: 'CCIV', 23: 'CBAT', 24: 'APT', 25: 'LUV', 26: 'QYLD', 27: 'SYK', 28: 'LPTX', 29: 'AA', 30: 'CFG', 31: 'JDST', 32: 'BLUE', 33: 'AEO', 34: 'YHOO', 35: 'SOXX', 36: 'OIH', 37: 'NIO', 38: 'NXPI', 39: 'GGG', 40: 'UPS', 41: 'SBGI', 42: 'WFC', 43: 'FAMI', 44: 'CVNA', 45: 'DIS', 46: 'MAG', 47: 'E', 48: 'ETRM', 49: 'CRTX', 50: 'GOOS', 51: 'XERS', 52: 'HPE', 53: 'BYND', 54: 'ARKK', 55: 'DLTR', 56: 'CS', 57: 'PAR', 58: 'SHAK', 59: 'KWEB', 60: 'GM', 61: 'RCRT', 62: 'CL', 63: 'SOS', 64: 'LOCO', 65: 'VZ', 66: 'IMCC', 67: 'XNET', 68: 'ZGNX', 69: 'CNY', 70: 'G', 71: 'MCMJ', 72: 'EVO', 73: 'DB', 74: 'LC', 75: 'CLPT', 76: 'GWSO', 77: 'DOCU', 78: 'TWLO', 79: 'NET', 80: 'A', 81: 'HLT', 82: 'ALDX', 83: 'SOFI', 84: 'INPX', 85: 'PINS', 86: 'DIA', 87: 'HOG', 88: 

In [8]:
DATA_PATH = '/content/data/content/idl-spring-22-project-deepar/data/'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [9]:
NUM_TRAIN_SAMPLES = 500000
NUM_TEST_SAMPLES  = 100000

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [10]:
del stock_inputs
del stock_labels
del stock_test_inputs
del stock_test_labels

In [11]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (500000, 192, 5)
stock_labels_trimmed shape (500000, 192)
stock_test_inputs_trimmed shape (100000, 192, 5)
stock_test_labels_trimmed shape (100000, 192)


In [12]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])




In [13]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

In [15]:

def generate_horizon_weights(horizon_len):


  # weight = [i for i in range(1, horizon_len + 1)]
  
  weight = [1]
  for i in range(1, horizon_len):
    weight.append(weight[-1]*2)

  weight_arr = np.array(weight)

  return weight_arr/ np.sum(weight_arr)

HORIZON_PERIOD = 8

params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':64,
    'cov_dim': 3,
    'lstm_hidden_dim': 256,
    'lstm_layers':4 ,
    'window_size':192,
    'batch_size': 32,
    'learning_rate': 2e-3,
    'epochs':30,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'horizon_period': HORIZON_PERIOD,
    'horizon_weights': generate_horizon_weights(HORIZON_PERIOD),
    'context_length':10
}

# print(hor_weight)

assert sum(params['horizon_weights']) == 1.0
assert len(params['horizon_weights']) == params['horizon_period']

print(params)

{'num_classes': 249, 'embedding_dim': 64, 'cov_dim': 3, 'lstm_hidden_dim': 256, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 32, 'learning_rate': 0.002, 'epochs': 30, 'num_test_samples': 100000, 'num_train_samples': 500000, 'conditioning_period': 168, 'prediction_period': 24, 'horizon_period': 8, 'horizon_weights': array([0.00392157, 0.00784314, 0.01568627, 0.03137255, 0.0627451 ,
       0.1254902 , 0.25098039, 0.50196078]), 'context_length': 10}


In [16]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys


class TrainDatasetContextHorizon(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        #Delete Volume
        self.data = np.delete(self.data, 1, axis=2)
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]
        self.horizon_period = params['horizon_period']  # Return (t+1, t+2, t+3, ...., t+5) into the future



    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index].astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))

        x_shift = [None]*self.horizon_period

        x_shift = [torch.roll(x2, shifts=-1*i, dims=0)[:-1*(self.horizon_period - 1 )] for i in range(0, self.horizon_period)]

        return x1[:-1*(self.horizon_period - 1 ), :], x_shift  

    
    def collate_fn(self, batch):
      batch_x =  []
      batch_y = [[None]*len(batch)] * self.horizon_period # [(B, SEQ_LEN)] * Horizon_period
      for batch_id, (x,y) in enumerate(batch):
        # print("y shape ", len(y), y[0].shape)

        for hor_ind in range(0, self.horizon_period):
            # curr_batch_val = batch_y[hor_ind]
            batch_y[hor_ind][batch_id] = y[hor_ind]

          

        opening_price = x[:,0].unsqueeze(1)
        open_prev = torch.zeros_like(opening_price)
        for i in range(1,params['context_length']):
          open_prev = torch.zeros_like(opening_price)
          open_prev[i:,:] = opening_price[:-i,:]
          x = torch.cat((x[:,:i],open_prev,x[:,i:]),dim=1)
        batch_x.append(x)
        # batch_y.append(y)

        # print("---batch_id---", batch_id, len(batch_y[batch_id]), len(batch_y))

      # print("batch y is ", batch_y)
      # print("len each batch size", len(batch_y[0]), "num batc", len(batch_y))
      ret_y_final = [torch.stack(batch_y[hor]) for hor in range(0, self.horizon_period)]

      return torch.stack(batch_x), ret_y_final
      # return torch.stack(batch_x),torch.stack(batch_y)

class TestDatasetContext(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        #Delete Volume
        self.data = np.delete(self.data, 1, axis=2)
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]

        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index].astype(np.float32))
      x2 = torch.from_numpy(self.label[index].astype(np.float32))
      return x1, x2

    def collate_fn(self, batch):
      batch_x,batch_y = [],[]
      for x,y in batch:

        opening_price = x[:,0].unsqueeze(1)
        open_prev = torch.zeros_like(opening_price)
        for i in range(1,params['context_length']):
          open_prev = torch.zeros_like(opening_price)
          open_prev[i:,:] = opening_price[:-i,:]
          x = torch.cat((x[:,:i],open_prev,x[:,i:]),dim=1)
        batch_x.append(x)
        batch_y.append(y)

      return torch.stack(batch_x),torch.stack(batch_y)

In [46]:

torch.cuda.empty_cache()

train_data = TrainDatasetContextHorizon()
train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True, collate_fn = train_data.collate_fn)


In [40]:
# del train_loader
# del train_data
# import gc
# gc.collect()

276

In [ ]:
# # del train_data
# # del train_loader
# # torch.cuda.empty_cache()

# # train_data = TrainDatasetContextHorizon()
# # train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True, collate_fn = train_data.collate_fn)


# for x, y_shift in train_loader:

#   print("---len x---", len(x), x.shape)
#   print("---len y---", len(y_shift), y_shift[0].shape)
#   # print("x shape", x, 'x shift', x_shift)

#   break

In [16]:
train_data = TrainDatasetContextHorizon()
test_data  = TestDatasetContext()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True, collate_fn = train_data.collate_fn)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False, collate_fn = test_data.collate_fn)

# train_data_unwindowed = unwindow_sequence(train_data, data_type="train_horizon")
# test_data_unwindowed = unwindow_sequence(test_data)


In [ ]:
# for x, y_shift in test_loader:
#   print("---len x---", len(x), x.shape)
#   print("---len y---", len(y_shift), y_shift[0].shape)

In [ ]:
# print("train_data_unwindowed", train_data_unwindowed.shape)
# print("test_data_unwindowed", test_data_unwindowed.shape)

In [17]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  # print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)

  B, SEQ_LEN, d = company_data.shape

  x=np.linspace(1, SEQ_LEN, num=SEQ_LEN)
  fig = px.line(x=x, y=company_data[window_id,:,covariate_index])
  fig.show()
  print("ip", company_data[window_id,:,covariate_index])


In [18]:
# plot_output_windowed(train_data, 0)

for x, y_shift in train_loader:
  print("---x 0 ---", x.shape)
  plot_input_covariate_windowed(x, 31, 0)
  plot_input_covariate_windowed(x, 31, 1)
  plot_input_covariate_windowed(x, 31, 2)

  break

---x 0 --- torch.Size([32, 185, 13])


ip tensor([-0.8141, -0.8230, -0.8654, -0.8654, -0.8440, -0.8810, -0.8779, -0.8654,
        -0.8290, -0.8200, -0.8053, -0.7880, -0.7458, -0.7186, -0.7458, -0.6921,
        -0.6791, -0.7485, -0.7880, -0.8230, -0.7822, -0.8083, -0.7458, -0.7652,
        -0.7851, -0.8083, -0.8410, -0.8937, -0.8748, -0.8319, -0.8748, -0.9033,
        -0.8937, -0.9129, -0.8748, -0.7851, -0.7596, -0.7822, -0.7458, -0.7709,
        -0.7513, -0.7541, -0.7458, -0.7430, -0.7737, -0.8410, -0.8623, -0.8562,
        -0.8873, -0.8685, -0.9097, -0.9065, -0.8531, -0.8905, -0.8562, -0.8654,
        -0.8470, -0.8562, -0.8593, -0.8654, -0.8531, -0.8440, -0.7937, -0.8053,
        -0.8112, -0.8260, -0.7966, -0.7937, -0.8410, -0.8380, -0.8410, -0.8440,
        -0.8380, -0.8141, -0.8260, -0.8349, -0.8531, -0.9097, -0.9097, -0.8531,
        -0.8905, -0.9162, -0.9194, -0.9458, -1.0186, -1.1085, -1.1607, -1.1731,
        -1.1164, -1.1525, -1.1444, -1.1444, -1.2288, -1.2830, -1.2200, -1.2691,
        -1.1857, -1.1484, -1.2157, -1

ip tensor([ 0.0000, -0.8141, -0.8230, -0.8654, -0.8654, -0.8440, -0.8810, -0.8779,
        -0.8654, -0.8290, -0.8200, -0.8053, -0.7880, -0.7458, -0.7186, -0.7458,
        -0.6921, -0.6791, -0.7485, -0.7880, -0.8230, -0.7822, -0.8083, -0.7458,
        -0.7652, -0.7851, -0.8083, -0.8410, -0.8937, -0.8748, -0.8319, -0.8748,
        -0.9033, -0.8937, -0.9129, -0.8748, -0.7851, -0.7596, -0.7822, -0.7458,
        -0.7709, -0.7513, -0.7541, -0.7458, -0.7430, -0.7737, -0.8410, -0.8623,
        -0.8562, -0.8873, -0.8685, -0.9097, -0.9065, -0.8531, -0.8905, -0.8562,
        -0.8654, -0.8470, -0.8562, -0.8593, -0.8654, -0.8531, -0.8440, -0.7937,
        -0.8053, -0.8112, -0.8260, -0.7966, -0.7937, -0.8410, -0.8380, -0.8410,
        -0.8440, -0.8380, -0.8141, -0.8260, -0.8349, -0.8531, -0.9097, -0.9097,
        -0.8531, -0.8905, -0.9162, -0.9194, -0.9458, -1.0186, -1.1085, -1.1607,
        -1.1731, -1.1164, -1.1525, -1.1444, -1.1444, -1.2288, -1.2830, -1.2200,
        -1.2691, -1.1857, -1.1484, -1

ip tensor([ 0.0000,  0.0000, -0.8141, -0.8230, -0.8654, -0.8654, -0.8440, -0.8810,
        -0.8779, -0.8654, -0.8290, -0.8200, -0.8053, -0.7880, -0.7458, -0.7186,
        -0.7458, -0.6921, -0.6791, -0.7485, -0.7880, -0.8230, -0.7822, -0.8083,
        -0.7458, -0.7652, -0.7851, -0.8083, -0.8410, -0.8937, -0.8748, -0.8319,
        -0.8748, -0.9033, -0.8937, -0.9129, -0.8748, -0.7851, -0.7596, -0.7822,
        -0.7458, -0.7709, -0.7513, -0.7541, -0.7458, -0.7430, -0.7737, -0.8410,
        -0.8623, -0.8562, -0.8873, -0.8685, -0.9097, -0.9065, -0.8531, -0.8905,
        -0.8562, -0.8654, -0.8470, -0.8562, -0.8593, -0.8654, -0.8531, -0.8440,
        -0.7937, -0.8053, -0.8112, -0.8260, -0.7966, -0.7937, -0.8410, -0.8380,
        -0.8410, -0.8440, -0.8380, -0.8141, -0.8260, -0.8349, -0.8531, -0.9097,
        -0.9097, -0.8531, -0.8905, -0.9162, -0.9194, -0.9458, -1.0186, -1.1085,
        -1.1607, -1.1731, -1.1164, -1.1525, -1.1444, -1.1444, -1.2288, -1.2830,
        -1.2200, -1.2691, -1.1857, -1

In [ ]:
print('train data shape', train_data[0][0].shape)

train data shape torch.Size([179, 4])


# MODEL


In [19]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim']+params['context_length']-1,
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )


        self.distribution_premu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_mu = nn.GELU()
        self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_sigma = nn.Softplus()


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        onehot_embed = self.embedding(company_index)
        
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim'] + params['context_length'] - 1
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim'] + params['context_length']-1

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_premu = self.distribution_premu(out1)
        out_mu = self.distribution_mu(out_premu)

        out_presigma = self.distribution_presigma(out1)
        out_sigma = self.distribution_sigma(out_presigma)

        return out_mu, out_sigma, hn_cn



In [20]:
model = Network(params)

In [21]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([32, 185, 1])
out_sigma shape torch.Size([32, 185, 1])
ht shape torch.Size([4, 32, 256]) torch.Size([4, 32, 256])


In [22]:
def loss_fn(mu,sigma,labels):

  batch,seq_len,mudim = mu.shape
  batch,seq_len,sigmadim = sigma.shape

  assert mudim==sigmadim

  mu = mu.view(batch*seq_len,mudim)
  sigma = sigma.view(batch*seq_len,sigmadim)
  labels = labels.view(batch*seq_len,1)

  gauss_prices = torch.distributions.Normal(mu,sigma)
  total_likelihood = -1 * torch.sum(gauss_prices.log_prob(labels))/(batch*seq_len)

  return total_likelihood

In [23]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  predictions = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [24]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [25]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


In [26]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [27]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [28]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period
      # print(cond_ip.shape) 
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last CONTEXT LENGTH VALUES of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -params['context_length']:, :], sigma[:, -1, :].unsqueeze(1)
      # print("pred_mu")
      # print(pred_mu)
      # print("pred_mu shape",pred_mu.shape)

      for t in range(0, prediction_period):

        #setting the covariates to the ones recieved from data loader
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        # print("initial input")
        # print(pred_cov_ip)

        #overwriting the opening price (t-1),(t-2),(t-3)
        for i in range(params['context_length']):
          # print("pred_cov_ip[:, 0, i]",pred_cov_ip[:, 0, i].shape)
          # print("pred_mu[:,:, 0]",pred_mu[:, :, 0].shape)
          pred_cov_ip[:, 0, i] = pred_mu[:, -(i+1), 0].squeeze()
        
        # print("overwritten input")
        # print(pred_cov_ip)
        
        mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        pred_mu = torch.roll(pred_mu, shifts=-1, dims=1)
        pred_mu[:,-1,:] = mu.squeeze(1)
        
        batch_mu = torch.cat((batch_mu,  mu.squeeze(1)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)

        


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
        
    rmse_val = rmse(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])
    nd_val = ND(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])

  return rmse_val,nd_val

In [29]:
# rmse = accuracy_RMSE_train(overall_mu, overall_label)
# print("rmse ", rmse)

In [30]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])


In [31]:
# model = Network(params)

validate(model)

(0.24219730185192478, 1.0016372)

# TRAIN

In [32]:
model_version='192winlen_weighted_loss_horizon10_v1.pt'
epochs = 50

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)

print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0

    if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
        # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

        checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch'] + 1

    for i, (ip_covariate, op_label_list) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        op_label_list     = [op_label.cuda() for op_label in op_label_list]

        mu, sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        loss = 0
        for i, op_label in enumerate(op_label_list):
          loss +=  loss_fn(mu, sigma, op_label) * params['horizon_weights'][i]

        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse, val_nd= validate(model=model)
    scheduler.step(val_rmse)

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
      torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'val_rmse': val_rmse,
              'loss': total_loss/len(train_loader),
              },  f'/content/drive/MyDrive/DeepARExperiments/best_dev_acc_{best_dev_rmse}_{model_version}')
    
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse, "nd is", val_nd)

Network(
  (embedding): Embedding(249, 128)
  (lstm): LSTM(140, 256, num_layers=4, batch_first=True)
  (distribution_premu): Linear(in_features=256, out_features=1, bias=True)
  (distribution_mu): GELU()
  (distribution_presigma): Linear(in_features=256, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


Train:  16%|█▌        | 2429/15625 [01:49<10:04, 21.82it/s, loss=68.7377, lr=0.0020]

KeyboardInterrupt: ignored

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/best_dev_acc_0.030142039248817844_deepar_model_no_volume_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:

def get_pred_sigma_mu_labels(model):

  model.eval()
  model.cuda()


  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)

  return overall_mu, overall_sigma, overall_label

In [ ]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, sigma, window_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [ ]:
# #s for the paper 3500 420
# model_no_volume = Network(params) 
# checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/deepar_model_no_volume_v1.pt')
# model_no_volume.load_state_dict(checkpoint['model_state_dict'])

model_no_volume = model

In [ ]:
# mu_no_volume, sigma_no_volume, label_no_volume =  get_pred_sigma_mu_labels(model)

# 420
# 5000
# 7000
# 11000
# 9900
# 9613
# 7105
# 2541
# 9632
# 63421
# 52132
# 57031
# 58031
# 67031
# 7031
# 9921
# 69000

plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 69000)

In [ ]:
rmse = accuracy_RMSE_test(mu_no_volume[:, 168:], label_no_volume[:, 168:])
print('rmse ', rmse)

rmse  0.14072188527960527


In [ ]:
# #s for the paper 3500 420 2500 69

# for i in range(0, 3500, 70):
plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 2550  )

In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result

print("rmse ", rmse(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

print("nd ", ND(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

rmse  0.06767744374120029
nd  0.057781804
